In [122]:
# -*- coding:utf-8 -*-
import json
import requests
import time
import pandas as pd
from decimal import *
import numpy as np

# 切分矩阵模块
def cut_div(l_lat, l_lng, r_lat, r_lng, size): # 左下经纬度，右上经纬度，切分尺寸
    l_lat = Decimal(str(l_lat))
    l_lng = Decimal(str(l_lng))
    r_lat = Decimal(str(r_lat))
    r_lng = Decimal(str(r_lng))
    size = Decimal(str(size))
    lat_list = np.arange(l_lat, r_lat, size)
    lng_list = np.arange(l_lng, r_lng, size)
    
    point_dict = dict()
    for lat in lat_list:
        for lng in lng_list:
            point_dict[str(lat) + ',' + str(lng)] = 1
    
    res_set = set()
    for k in point_dict:
        r_lat = str(Decimal(k.split(',')[0]) + Decimal(size))
        r_lng = str(Decimal(k.split(',')[1]) + Decimal(size))
        k1 = r_lat + ',' + r_lng
        if k1 in point_dict:
            res_set.add(k + ',' + k1)
    return res_set

# 调用百度API模块
def query_baidu(bounds, page_num):
    time.sleep(1) # 避免百度单秒API调用次数限制
    pa = {
        'query': query,
        'bounds': bounds,
        'page_size': 20,
        'page_num': page_num,
        'output': 'json',
        'ak': ak
    }
    r = requests.get('http://api.map.baidu.com/place/v2/search?', params=pa)
    decodejson = json.loads(r.text)
    try:
        for item in decodejson['results']:
            item['lat'] = str(item['location']['lat'])
            item['lng'] = str(item['location']['lng'])
            poi_list.append(item)
        total = decodejson['total'] # 总poi数量
        if ((page_num+1)*20) < total: # 没查完就再翻下一页 
            query_baidu(bounds, page_num+1)
    except Exception as e:
        print(e)

# 循环网格搜索poi
print("开始爬取数据，请稍等...")
start_time = time.time()

bounds_list = cut_div(30.653785, 120.857029, 31.859884, 122.040205, 1.0) # 修改矩形坐标, 切分尺寸
query = '美食' # 修改查询的行业
ak = 'foDFvFh35NHc7sFaPjzp6SYLjV3fMCN7' # 修改百度api key

poi_list = []
for bounds in bounds_list:
    query_baidu(bounds, 0) # 每一格从第一页开始查询 
    
df = pd.DataFrame(poi_list)
df.to_csv('导出数据.csv', encoding='utf_8_sig')

end_time = time.time()
print("爬取完毕，用时%.2f秒" % (end_time - start_time))

开始爬取数据，请稍等...
爬取完毕，用时9.09秒


In [ ]:
# 导入需要的包
import pandas as pd
import numpy as np
import requests
import json

# 多城市、多类型查询
poi_list = []
ak = 'foDFvFh35NHc7sFaPjzp6SYLjV3fMCN7' # 改成你自己的百度地图api key
def get_poi_list(region, query, page_num):
    global poi_list
    pa = {
        'query': query, #必选
        'region': region, #必选
        'city_limit': 'false',
        'extensions_adcode': 'true',
        'output': 'json',
        'scope': '1',
        'coord_type': 1, #wgs84ll即GPS经纬度
        'page_size': 20,
        'page_num': page_num, #0代表第一页
        'ak': ak, #必选
    }
    r = requests.get('http://api.map.baidu.com/place/v2/search?', params=pa)
    decodejson = json.loads(r.text)
    print(decodejson)
    try:
        pois = decodejson['results']
        if(len(pois)):
            poi_list += pois
            get_poi_list(region, query, page_num+1) # 递归
        else: 
            return
    except Exception as e:
        print(e)

# 每次在这里修改城市、一级行业分类
region = '长沙'
query_list = [
    '美食',
    #'酒店',
    #'购物',
    #'生活服务',
    #'丽人',
    #'旅游景点',
    #'休闲娱乐',
    #'运动健身',
    #'教育培训',
    #'文化传媒',
    #'医疗',
    #'汽车服务',
    #'交通设施',
    #'金融',
    #'房地产',
    #'公司企业',
    #'政府机构',
    #'出入口',
    #'自然地物',
    #'行政坐标',
    #'门址',
]
for query in query_list:
    get_poi_list(region, query, 0)
df = pd.DataFrame(poi_list)
df.to_excel(region + '_poi.xlsx')